# Import data

In [22]:
from Utils.Toolkit import TestGen, DataReader
from Utils.Toolkit import TestSplit
dataReader = DataReader()

# Generate Test & Setup

In [23]:
k_fold = TestGen(dataReader.URM_CSR(), TestSplit.K_FOLD, k=3)
matrices = k_fold.get_k_fold_matrices()
target_users = dataReader.targetUsersList

# ItemCF K_Fold

In [24]:
from CF.item_cf import ItemBasedCollaborativeFiltering

for i in range(len(matrices)):
    test_matrix = matrices[i]
    train_matrix = None

    for j in range(len(matrices)):
            if j != i:
                if train_matrix is not None:
                    train_matrix += matrices[j]
                else:
                    train_matrix = matrices[j]

    itemCF1 = ItemBasedCollaborativeFiltering(train_matrix, topK=10, shrink=30)
    itemCF1.fit(similarity="tanimoto")
    itemCF1.evaluate_MAP_target(test_matrix, target_users)

UserCF -> MAP: 0.0428 with TopK = 10 & Shrink = 30	
UserCF -> MAP: 0.0427 with TopK = 10 & Shrink = 30	
UserCF -> MAP: 0.0426 with TopK = 10 & Shrink = 30	


# UserCF K_Fold

In [25]:
from CF.user_cf import UserBasedCollaborativeFiltering

for i in range(len(matrices)):
    test_matrix = matrices[i]
    train_matrix = None

    for j in range(len(matrices)):
            if j != i:
                if train_matrix is not None:
                    train_matrix += matrices[j]
                else:
                    train_matrix = matrices[j]

    userCF1 = UserBasedCollaborativeFiltering(train_matrix, topK=100, shrink=500)
    userCF1.fit()
    userCF1.evaluate_MAP_target(test_matrix, target_users)

UserCF -> MAP: 0.0334 with TopK = 100 & Shrink = 500	, elapsed time 0.30 min
UserCF -> MAP: 0.0324 with TopK = 100 & Shrink = 500	, elapsed time 0.30 min
UserCF -> MAP: 0.0336 with TopK = 100 & Shrink = 500	, elapsed time 0.32 min


# Hybrid K_Fold

In [26]:
from HYB.hybrid import HybridRecommender

for i in range(len(matrices)):
    test_matrix = matrices[i]
    train_matrix = None

    for j in range(len(matrices)):
            if j != i:
                if train_matrix is not None:
                    train_matrix += matrices[j]
                else:
                    train_matrix = matrices[j]

    hyb = HybridRecommender(train_matrix, userCF1, itemCF1)
    hyb.fit(1, 2)
    hyb.evaluate_MAP_target(test_matrix, target_users)

TypeError: __init__() takes 2 positional arguments but 4 were given